In [ ]:
from plots.plot_utils import load_pickle
data = load_pickle("boston_lossscaled_batch_cal_ens1_bootFalse_seed0_thres0.01-0.15.pkl")

In [ ]:
for entry in zip(data['te_ece_list_best'], data['te_sharp_score_list_best']):
    print(entry)

In [ ]:
from utils.misc_utils import EceSharpFrontier
frontier = EceSharpFrontier.from_list(list(zip(data['te_ece_list_best'], data['te_sharp_score_list_best'])), only_frontier=True)

In [ ]:
thresholded_frontiers = frontier.get_thresholded_frontier()
import matplotlib.pyplot as plt
# [{'ece': 0.07363636363636362, 'sharp': 0.0935474103877344, 'model': None},
# {'ece': 0.07505050505050503, 'sharp': 0.0935474103877344, 'model': None},
ece_list = [entry['ece'] for entry in thresholded_frontiers]
sharp_list = [entry['sharp'] for entry in thresholded_frontiers]

plt.scatter(ece_list, sharp_list)

In [ ]:
for entry in zip(data['thresholds'], data['va_ece_list_best'], data['va_sharp_score_list_best']):
    print(entry)

In [ ]:
len(data['thresholds'])

In [ ]:
import glob
pkls = glob.glob("results/boston*.pkl")
from plots.plot_utils import load_pickle 
objs = []
for pkl in pkls:
    data = load_pickle(pkl)
    objs.append((data, str(pkl)))


In [ ]:
len(objs[0])

In [ ]:
ece_sharpness = []
for obj, pkl in objs:
    ece_sharpness.append((obj['te_ece_list_best'], obj['te_sharp_score_list_best'], pkl))

In [ ]:
len(ece_sharpness)

In [ ]:
import matplotlib.pyplot as plt

for ece, sharpness, pkl in ece_sharpness:
    plt.plot(ece, sharpness, label=pkl)
    plt.title("Boston ECE vs Sharpness")
    plt.legend(loc="lower center")

In [ ]:
from plots.plot_metrics import overlap_ece_sharpness
from plots.plot_utils import load_pickle
import os
results_dir = "results"
names = ["batch_qr", "batch_int", "batch_cal"]
example_pkl = "kin8nm_lossbatch_cal_ens1_bootFalse_resFalse_lnFalse_bnFalse_dr0.0_lr0.001_bs64.pkl"
example_pkls = ["{}/{}".format(results_dir, example_pkl.replace("batch_cal", name)) for name in names]
for pkl in example_pkls:
    datas = [load_pickle(pkl) for pkl in example_pkls]
    overlap_ece_sharpness(datas, example_pkls, show=True)

In [18]:
from plots.plot_utils import load_pickle
from data.fetch_data import get_uci_data
from argparse import Namespace
data = load_pickle("/home/scratch/yixiz/test/boston_lossbatch_cal_ens1_bootFalse_resFalse_lnFalse_bnFalse_dr0.0_lr0.001_bs64_nl2_hs64.pkl")
args = Namespace(data_dir="data/UCI_Datasets", data="boston", seed=0)
dataset = get_uci_data(args)

In [26]:
import torch
y_al = dataset.y_al
x_tr, x_va, x_te, y_tr, y_va, y_te, y_al = (
    dataset.x_tr,
    dataset.x_va,
    dataset.x_te,
    dataset.y_tr,
    dataset.y_va,
    dataset.y_te,
    dataset.y_al,
)
y_range = torch.max(y_al) - torch.min(y_al)
y_range

tensor(4.9238)

In [42]:
example_model = data['model_controlled'][0]
quantiles = torch.tensor([0.02, 0.97])
intervals = example_model.predict_q(x_te, quantiles)

In [45]:
interval_lengths = intervals[:, 1] - intervals[:, 0]
interval_lengths.mean() / y_range

tensor(0.0949)

In [47]:
data['te_sharp_score_controlled'][0]

0.09493224182174952